In [1]:
pwd

'C:\\Users\\Admin\\Desktop\\R projects\\Spot-the-Mask-Challenge-by-ZindiWeekendz-master'

In [2]:
!pip install utils

You should consider upgrading via the 'c:\users\admin\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
import os, cv2, glob
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten, BatchNormalization, Input
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import os
import utils
import random
import shutil # operation on files from directories
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [4]:
# load data
image_dir = "./images/"
train = pd.read_csv('Train_Labels_Dataset.csv')
sample_sub = pd.read_csv('Sample_Submission_File.csv')

In [5]:
train.head()

,image,target
0,tfymlmdkpzkqdjhdxyhnoeuqszxphw.jpg,0
1,rxgismgsvmaayzjarbfjaljhqmpbrt.jpg,1
2,uuzshfrhkgrkolhwdvliqauzulurnz.jpg,0
3,mjspxsagzusaznvnyxgamtrlqkqklp.jpg,0
4,rlbmuajgezfiddjzlyeoupxpqubkpt.jpg,1


In [6]:
# create image directory for train and test loaded into img_dir workspace
dirs = ['train_imgs', 'test_imgs'] #folders to be created
all_files = './' # current directory path
for files in dirs:
    os.makedirs(os.path.join(all_files, files)) # creates the two files in the current directory

In [7]:
# split images and data into training and test set
#get list of all files and drectories in the image_dir
img_names = os.listdir(image_dir) #list of all files(images)
#
# split the data into train and test
trn_names = train.image.tolist() #returns train image names and convert them to a list
tst_names = [] # empty list to store test image names
#
for img in img_names:
    if img not in trn_names:
        tst_names.append(img)

In [8]:
# move the train and test images to their respective directories
srcdir = './images'
for filePath in glob.glob(srcdir + '/*'):
            # Move each file to destination Directory
            if os.path.basename(filePath) in trn_names: # grabs the base name that belongs to the names image list
                shutil.move(filePath, './train_imgs')
            else:
                shutil.move(filePath, './test_imgs')

In [9]:
trndir = './train_imgs'
d = ['mask-on', 'mask-off'] #folders to be created
train_files = './train_imgs/' # train images directory path
#
for files in d:
    os.makedirs(os.path.join(train_files, files))
#
for filep in glob.glob(trndir + '/*'):
    if os.path.basename(filep) in train[train['target'] == 0].drop('target', axis = 1).image.tolist():
        shutil.move(filep, './train_imgs/mask-off')
    else:
        shutil.move(filep, './train_imgs/mask-on')

In [12]:
# global variables
random.seed(1234)
img_width, img_height = 128, 128
channels = 3
#
# reshape the inputs
from keras.layers import Dropout
from keras import backend as K 
if K.image_data_format() == 'channels_first': 
    input_shape = (channels, img_width, img_height) 
else: 
    input_shape = (img_width, img_height, channels) 
#
# algorithm
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128, (3,3), activation='relu'))
#
#  layer flatten and dense
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()
#
# compile the CNN algorithm
model.compile(loss ='binary_crossentropy', 
              optimizer ='adam', 
              metrics =['accuracy']) 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 12, 12, 128)      

In [13]:
# fit the CNN to target images
train_datagen = ImageDataGenerator(rescale = 1. / 255, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True) 
#
train_path = 'C:/Users/Admin/Desktop/R projects/Spot-the-Mask-Challenge-by-ZindiWeekendz-master/train_imgs'
training_gen = train_datagen.flow_from_directory(train_path,
                                                 target_size = (img_width, img_height),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 954 images belonging to 2 classes.


In [15]:
test_datagen = ImageDataGenerator(rescale = 1. / 255)
#
test_path = 'C:/Users/Admin/Desktop/R projects/Spot-the-Mask-Challenge-by-ZindiWeekendz-master/test_imgs'
test_gen = test_datagen.flow_from_directory('.', classes = ['test_imgs'],
                                            target_size = (img_width, img_height),
                                            batch_size = 32, #no. of samples to run the algorithm through 
                                            class_mode = 'binary')

Found 509 images belonging to 1 classes.


In [17]:
model.fit_generator(training_gen,
                         samples_per_epoch = 8000,
                         nb_epoch = 10,#no. of complete passes/complete algo loops
                         validation_data = test_gen,
                         nb_val_samples = 20)

Epoch 1/10
250/250 [==============================] - 314s 1s/step - loss: 0.2949 - accuracy: 0.8702 - val_loss: 3.7358 - val_accuracy: 0.3469
Epoch 2/10
250/250 [==============================] - 235s 940ms/step - loss: 0.1874 - accuracy: 0.9211 - val_loss: 3.7904 - val_accuracy: 0.4097
Epoch 3/10
250/250 [==============================] - 236s 945ms/step - loss: 0.1167 - accuracy: 0.9575 - val_loss: 2.6083 - val_accuracy: 0.4223
Epoch 4/10
250/250 [==============================] - 233s 932ms/step - loss: 0.0917 - accuracy: 0.9671 - val_loss: 5.7507 - val_accuracy: 0.4054
Epoch 5/10
250/250 [==============================] - 232s 928ms/step - loss: 0.0617 - accuracy: 0.9768 - val_loss: 5.8942 - val_accuracy: 0.4380
Epoch 6/10
250/250 [==============================] - 233s 933ms/step - loss: 0.0519 - accuracy: 0.9809 - val_loss: 10.8669 - val_accuracy: 0.3564
Epoch 7/10
250/250 [==============================] - 234s 935ms/step - loss: 0.0332 - accuracy: 0.9886 - val_loss: 11.0290 - 

In [44]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
 
# load and prepare the image
def load_image(filename):
    # load the image
    img = load_img(filename, target_size=(128, 128))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = img.reshape(1, 128, 128, 3)
    # center pixel data
    img = img.astype('float32')
    img = img - [123.68, 116.779, 103.939]
    return img
#
# load an image and predict the class
def describe_image():
    for image in glob.glob('./test_imgs/*'): 
        # load the image
        img = load_image(image)
        # predict the class
        result = model.predict(img)
        print(result[0])
describe_image()

[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
